In [1]:
import torch

torch.__version__

'2.3.1+cu118'

In [3]:
scalar = torch.tensor(5)

scalar

tensor(5)

In [4]:
scalar.ndim

0

In [5]:
scalar.item()

5

In [6]:
a = torch.tensor([1,2,3], dtype=torch.int8)

In [7]:
a

tensor([1, 2, 3], dtype=torch.int8)

In [8]:
a[0] = 5

In [11]:
# test overflow
try:
  a[0] = 300
except RuntimeError:
  print('overflow')


overflow


In [14]:
a @ a

tensor(38, dtype=torch.int8)

In [18]:
a = torch.randn(2, 3)
b = torch.randn(2, 3)

a @ b.T

tensor([[ 1.4234, -0.1143],
        [ 0.5270, -0.6216]])

In [19]:
a.T @ b

tensor([[ 0.7803,  0.6673, -0.1511],
        [-1.0524, -1.1039,  0.1051],
        [-2.9647, -1.3949,  1.1254]])

In [20]:
a * b

tensor([[ 0.4044, -0.1256,  1.1446],
        [ 0.3760, -0.9783, -0.0193]])

# Test Linear

In [27]:
x = torch.rand(10, 10)
b = torch.rand(10, 1)
a = torch.rand(10, 1)

y =  x @ a + b

y


tensor([[2.8888],
        [2.9941],
        [3.8659],
        [2.3848],
        [3.0561],
        [4.0631],
        [3.0783],
        [3.3534],
        [2.3534],
        [2.8407]])

In [72]:
x = torch.rand(1, 10)

In [73]:
layers = [
  torch.nn.Linear(10, 10),
  torch.nn.Linear(10, 5),
  torch.nn.Linear(5, 2)
]

model = torch.nn.Sequential(*layers)

model(x)

tensor([[ 0.0486, -0.1841]], grad_fn=<AddmmBackward0>)

In [74]:
from functools import partial

In [75]:
def my_linear(w: torch.Tensor,b: torch.Tensor, x: torch.Tensor) -> torch.tensor:
  print(w.shape, x.shape)
  return x @ w.T + b

my_lin = [partial(my_linear,layer.weight, layer.bias) for layer in layers]

for f in my_lin:
  x = f(x)
  print(x.shape)


torch.Size([10, 10]) torch.Size([1, 10])
torch.Size([1, 10])
torch.Size([5, 10]) torch.Size([1, 10])
torch.Size([1, 5])
torch.Size([2, 5]) torch.Size([1, 5])
torch.Size([1, 2])


In [76]:
x

tensor([[ 0.0486, -0.1841]], grad_fn=<AddBackward0>)

# Activation test

In [95]:
x = torch.rand(1, 10)

sigmoid = torch.nn.Sigmoid()

sigmoid(x)

tensor([[0.5507, 0.6566, 0.7220, 0.6353, 0.6548, 0.5035, 0.6156, 0.7106, 0.5959,
         0.6697]])

In [98]:
def my_sigmoid(x: torch.Tensor) -> torch.Tensor:
  return torch.tensor([torch.exp(val)/ (1+torch.exp(val)) if val > torch.tensor(0) else 1/(1+torch.exp(-x)) for val in x.squeeze()])
my_sigmoid(x)

tensor([0.5507, 0.6566, 0.7220, 0.6353, 0.6548, 0.5035, 0.6156, 0.7106, 0.5959,
        0.6697])

In [99]:
softmax = torch.nn.Softmax(dim=1)

softmax(x)

tensor([[0.0683, 0.1065, 0.1447, 0.0971, 0.1057, 0.0565, 0.0892, 0.1368, 0.0822,
         0.1130]])

In [105]:
def my_softmax(x: torch.Tensor) -> torch.Tensor:
  return torch.tensor([torch.exp(val)/torch.sum(torch.exp(x)) for val in x.squeeze(0)])

my_softmax(x)

tensor([0.0683, 0.1065, 0.1447, 0.0971, 0.1057, 0.0565, 0.0892, 0.1368, 0.0822,
        0.1130])

In [107]:
class LinearRegression(torch.nn.Module):
  def __init__(self):
    super(LinearRegression, self).__init__()
    
    self.weight = torch.nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
    self.bias = torch.nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float)) 
    
  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.weight * x + self.bias
  
    

In [120]:
model = LinearRegression().eval()

input = (torch.rand(1),)

ex = torch.export.export(model, input)





In [122]:
torch.export.save(ex, 'model.pt2')

In [123]:
from torch import nn

: 

In [2]:
class Unet(nn.Module):
  def __init__(self):
    self.contracting_1 = nn.Sequential([
      nn.Conv2d(in_channels=3, out_channels=64),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.Conv2d(in_channels=3, out_channels=64),
      nn.BatchNorm2d(),
      nn.ReLU(),
    ])
    self.contracting_2 = nn.Sequential([
      nn.MaxPool2d((2, 2), stride=2),
      nn.Conv2d(in_channels=64, out_channels=128),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.Conv2d(in_channels=64, out_channels=128),
      nn.BatchNorm2d(),
      nn.ReLU(),
    ])
    self.contracting_3 = nn.Sequential([
      nn.MaxPool2d((2, 2), stride=2),
      nn.Conv2d(in_channels=128, out_channels=256),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.Conv2d(in_channels=128, out_channels=256),
      nn.BatchNorm2d(),
      nn.ReLU(),
    ])
    self.contracting_4 = nn.Sequential([
      nn.MaxPool2d((2, 2), stride=2),
      nn.Conv2d(in_channels=256, out_channels=512),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.Conv2d(in_channels=256, out_channels=512),
      nn.BatchNorm2d(),
      nn.ReLU(),
    ])
    self.contracting_5 = nn.Sequential([
      nn.MaxPool2d((2, 2), stride=2),
      nn.Conv2d(in_channels=512, out_channels=1024),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.Conv2d(in_channels=512, out_channels=1024),
      nn.BatchNorm2d(),
      nn.ReLU(),
    ])
    
    self.expansive_1 = nn.Sequential([
      nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True),
      nn.Conv2d(in_channels=1024, out_channels=512),
      nn.BatchNorm2d(),
      nn.ReLU(),
      nn.Conv2d(in_channels=1024, out_channels=512),
      nn.BatchNorm2d(),
      nn.ReLU(),
    ])
    

    
    
    
    

SyntaxError: invalid syntax (804892183.py, line 42)

: 